In [1]:
import numpy as np
from sklearn.manifold import TSNE
import pickle
# import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import time
import sqlite3
import pandas as pd
import json

In [2]:
song_id_path = "./data/song_track_artist.pkl"

# Get genre embedding latent vectors

In [3]:
genre_embeddings_path = "./data/latent_embeddings.npy"
track_id_embeddings_path = "./data/track_ids_nn.txt"

In [4]:
genre_embeddings_np = np.load(genre_embeddings_path)
track_ids = np.loadtxt(track_id_embeddings_path, dtype="str")
genre_embeddings_duplicates = pd.DataFrame(genre_embeddings_np, columns=list(range(genre_embeddings_np.shape[1])))
genre_embeddings_duplicates.index = track_ids

In [5]:
genre_embeddings = genre_embeddings_duplicates[~genre_embeddings_duplicates.index.duplicated(keep='first')]

# get genre

In [6]:
len(better_df)

NameError: name 'better_df' is not defined

In [7]:
with open("./data/track-genre-mapping.json", "r") as f:
    track_genre = json.load(f)

# Set track id filter to filtered_track_ids (used throughout)

In [8]:
filtered_track_ids = genre_embeddings.index[:3000]

# Get song_id to song_name mapping


### using pkl file

In [9]:
with open(song_id_path, "rb") as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    song_name_artist = u.load()

In [10]:
# getting everything in the right order
song_name_artist_ordered = song_name_artist.set_index(["track_id"]).loc[filtered_track_ids]
# song_artist_str = song_name_artist_ordered.apply(lambda r: "artist: " + r["artist_id"] + ", song: " + r["song_name"], axis=1)
song_artist_str = song_name_artist_ordered["artist_id"]




In [11]:
song_name_artist_ordered.shape

(2921, 3)

In [12]:
artists = song_name_artist_ordered["artist_id"].values

# Mark artists

In [13]:
famous = ["Lady GaGa", "Kanye West", "Bob Marley", "Bob Dylan", "Shakira", "Jay-Z", "Bon Jovi", "Michael Jackson", 
         "Mariah Carey", "Justin Timberlake", "Oasis"]

In [14]:
# colors = [int(a in famous) for a in artists]

In [15]:
# max_popularity = song_popularity.iloc[0].values[0]
# pre_colors = np.array([x[0]/max_popularity for x in song_popularity.values])

In [16]:
genres = list(set(track_genre.values()))
genres

['Vocal',
 'Punk',
 'Rock',
 'Country',
 'Blues',
 'New',
 'World',
 'Reggae',
 'Jazz',
 'Folk',
 'RnB',
 'International',
 'Pop_Rock',
 'Electronic',
 'Metal',
 'Rap',
 'Pop',
 'Latin']

In [17]:
genres.index("Blues")

4

In [18]:
def foo(t):
    global genres
    if t in track_genre:
        return genres.index(track_genre[t])
    return -1
colors = song_name_artist_ordered.index.map(foo).values

In [19]:
colors

array([2, 2, 2, ..., 2, 2, 2])

In [20]:
for a in artists:
    if "Kurt" in a:
        print(a)

In [21]:
colors

array([2, 2, 2, ..., 2, 2, 2])

# Create t-SNE

In [22]:
filtered_track_ids

Index(['TRMMMTK128F424EF7C', 'TRMMMKQ128F92EBCB5', 'TRMMMFG128F425087B',
       'TRMMMQA128F14A454A', 'TRMMWJS12903CBB7F5', 'TRMMWQU12903CF447F',
       'TRMMGTX128F92FB4D9', 'TRMMHDI128F42442F4', 'TRMMHKG12903CDB1B5',
       'TRMMCBU128E0791823',
       ...
       'TRGQXUP128F92FFC1D', 'TRGQXDF12903D0FB07', 'TRGQXRC128E0780ED4',
       'TRGQKVS12903CF5E1C', 'TRGQKTS128F428913E', 'TRGQYEQ12903CDFB2B',
       'TRGQYAK128F9351C6B', 'TRGQYOV128F92DF473', 'TRGZMFZ128F427F03B',
       'TRGZWLW128F9309DE8'],
      dtype='object', length=2921)

In [23]:
embeddings_pure = genre_embeddings.loc[filtered_track_ids].values

In [24]:
embeddings_pure.shape

(2921, 50)

In [25]:
X_embedded = TSNE(n_components=2).fit_transform(embeddings_pure)
X_embedded.shape

(2921, 2)

In [28]:
by_genre_factors = {i: X_embedded[i == colors[:len(X_embedded)]] for i in range(len(genres))}
by_genre_text = {i: song_artist_str[i == colors] for i in range(len(genres))}

In [29]:
import colorlover as cl

In [30]:
bupu = cl.scales['11']['div']['RdYlBu']

bupu500 = cl.interp( bupu, 18 )

In [34]:
# create a new plot with a title and axis labels
scale = [[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)'], [0.2222222222222222, 'rgb(244,109,67)'],
        [0.3333333333333333, 'rgb(253,174,97)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'],
        [0.6666666666666666, 'rgb(171,217,233)'],[0.7777777777777778, 'rgb(116,173,209)'], [0.8888888888888888, 'rgb(69,117,180)'],
        [1.0, 'rgb(49,54,149)']]

data = [go.Scatter(x=by_genre_factors[i][:,0], y=by_genre_factors[i][:,1], mode='markers',
    marker=dict(
        size=7,
        color=bupu500[i]
    ), text=np.array(by_genre_text[i]), name=genres[i]) for i in range(len(genres))]

# fig = {
#     'data': [
#         {
#             'x': df[df['year']==year]['gdpPercap'],
#             'y': df[df['year']==year]['lifeExp'],
#             'name': year, 'mode': 'markers',
#         } for year in [1952, 1982, 2007]
#     ],
#     'layout': {
#         'xaxis': {'title': 'GDP per Capita', 'type': 'log'},
#         'yaxis': {'title': "Life Expectancy"}
#     }
# }


plotly.offline.plot(data, filename="tsne-scatter-genre.html")

'tsne-scatter-genre.html'

In [ ]:
 cl.scales['11']

In [25]:
!pip install --user colorlover 

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
